In [8]:
import urllib.request
import urllib
import pandas as pd
import re
from bs4 import BeautifulSoup

In [20]:
def scrape():
    urlRC =  'http://www.beefitswhatsfordinner.com/recipes/collection'
    recipeCollection = [urlRC + '/10002/grilling-favorites',
                        urlRC + '/10008/not-your-average-salad',
                        urlRC + '/10009/30-minutes-or-less',
                        urlRC + '/10011/appetizers-worth-sharing',
                        urlRC + '/10014/flavor-boosting-rubs-marinades',
                        urlRC + '/10018/well-balanced-meals',
                        urlRC + '/10030/tailgating-treasures',
                        urlRC + '/10032/slow-cooker-comforts',
                        urlRC + '/10033/so-long-leftovers',
                        urlRC + '/10034/recipes-for-a-crowd',
                        urlRC + '/10039/the-tastiest-burgers',
                        urlRC + '/10044/savory-stir-fry',
                        urlRC + '/10045/meals-in-the-skillet',
                        urlRC + '/10049/more-soup-for-you',
                        urlRC + '/10052/from-the-deli',
                        urlRC + '/10058/kid-friendly-fare',
                        urlRC + '/10061/gotta-try-beef-chilis',
                        urlRC + '/10070/bbq-for-you']
    recipeList = findHTML(recipeCollection)
    rawDataList = createDF(recipeList)
    
    rawData = pd.DataFrame(
        {'url': rawDataList[0],
         'name': rawDataList[1],
         'ingredient': rawDataList[2]
        })
    rawData.to_csv('SUN_info370_assignment1_rawData.csv')

In [16]:
# urlMain = 'http://www.beefitswhatsfordinner.com'
# urlRC =  'http://www.beefitswhatsfordinner.com/recipes/collection'
# recipeList = []

# recipeCollection = [urlRC + '/10002/grilling-favorites',
#                     urlRC + '/10008/not-your-average-salad',
#                     urlRC + '/10009/30-minutes-or-less',
#                     urlRC + '/10011/appetizers-worth-sharing',
#                     urlRC + '/10014/flavor-boosting-rubs-marinades',
#                     urlRC + '/10018/well-balanced-meals',
#                     urlRC + '/10030/tailgating-treasures',
#                     urlRC + '/10032/slow-cooker-comforts',
#                     urlRC + '/10033/so-long-leftovers',
#                     urlRC + '/10034/recipes-for-a-crowd',
#                     urlRC + '/10039/the-tastiest-burgers',
#                     urlRC + '/10044/savory-stir-fry',
#                     urlRC + '/10045/meals-in-the-skillet',
#                     urlRC + '/10049/more-soup-for-you',
#                     urlRC + '/10052/from-the-deli',
#                     urlRC + '/10058/kid-friendly-fare',
#                     urlRC + '/10061/gotta-try-beef-chilis',
#                     urlRC + '/10070/bbq-for-you']

['http://www.beefitswhatsfordinner.com/recipes/recipe/656/grecian-beef-strip-steaks-mushroom-kabobs', 'http://www.beefitswhatsfordinner.com/recipes/recipe/741/grilled-t-bone-steaks-with-bbq-rub', 'http://www.beefitswhatsfordinner.com/recipes/recipe/975/firecracker-burgers-with-cooling-lime-sauce', 'http://www.beefitswhatsfordinner.com/recipes/recipe/1031/balsamic-marinated-flank-steak', 'http://www.beefitswhatsfordinner.com/recipes/recipe/1080/burgers-with-grilled-onions', 'http://www.beefitswhatsfordinner.com/recipes/recipe/1104/mushroom-merlot-burger', 'http://www.beefitswhatsfordinner.com/recipes/recipe/1332/spicy-beef-back-ribs', 'http://www.beefitswhatsfordinner.com/recipes/recipe/1632/ribeye-steaks-with-blue-cheese-butter-and-mushrooms', 'http://www.beefitswhatsfordinner.com/recipes/recipe/1685/garden-herb-strip-steaks', 'http://www.beefitswhatsfordinner.com/recipes/recipe/1926/chimichurri-marinated-strip-filets', 'http://www.beefitswhatsfordinner.com/recipes/recipe/402/confetti-

In [18]:
def findHTML(recipeCollection):
    recipeList = []
    urlMain = 'http://www.beefitswhatsfordinner.com'
    for page in recipeCollection:
        with urllib.request.urlopen(page) as url:
            page = url.read()  
            parsed_page = BeautifulSoup(page, 'lxml')
            menu_box = parsed_page.findAll('div', attrs={'class': 'col-sm-4 col-md-6 recipe-img'})
            for div in menu_box:
                links = div.findAll('a')
                for a in links:
                    try:
                        recipeList.append(urlMain + a['href'])
                    except KeyError:
                        pass
    return recipeList

In [4]:
# for page in recipeCollection:
#     with urllib.request.urlopen(page) as url:
#         page = url.read()  
#         parsed_page = BeautifulSoup(page, 'lxml')
#         menu_box = parsed_page.findAll('div', attrs={'class': 'col-sm-4 col-md-6 recipe-img'})
#         for div in menu_box:
#             links = div.findAll('a')
#             for a in links:
#                 try:
#                     recipeList.append(urlMain + a['href'])
#                 except KeyError:
#                     pass

In [19]:
def createDF(recipeList):    
    ingredientList = []
    recipeName = []
    recipeURL = []

    #scrapes each link which contains a recipe for ingredients and recipe name
    for link in recipeList:
        with urllib.request.urlopen(link) as url:
            page = url.read()  
            parsed_page = BeautifulSoup(page, 'lxml')
            name = ''

            #grabs the title of the current recipe
            menu_title = parsed_page.findAll('li', attrs={'class': 'current'})
            for div in menu_title:
                name = div.text.strip()

            #grabs all of the ingredients in the current recipe
            menu_box = parsed_page.findAll('li', attrs={'itemprop': 'ingredients'})
            for div in menu_box:

                #removes all words after a comma
                item = div.text.strip().split(',', 1)[0]

                #removes the word '(optional)' which appears next to some ingredients
                item = item.split('(', 1)[0]

                #add url, recipe, and ingredient to their corresponding list
                ingredientList.append(item.lower())
                recipeURL.append(link.lower())
                recipeName.append(name.lower())
                
    return [ingredientList, recipeName, recipeURL]

In [5]:
# ingredientList = []
# recipeName = []
# recipeURL = []

# #scrapes each link which contains a recipe for ingredients and recipe name
# for link in recipeList:
#     with urllib.request.urlopen(link) as url:
#         page = url.read()  
#         parsed_page = BeautifulSoup(page, 'lxml')
#         name = ''
        
#         #grabs the title of the current recipe
#         menu_title = parsed_page.findAll('li', attrs={'class': 'current'})
#         for div in menu_title:
#             name = div.text.strip()
            
#         #grabs all of the ingredients in the current recipe
#         menu_box = parsed_page.findAll('li', attrs={'itemprop': 'ingredients'})
#         for div in menu_box:
            
#             #removes all words after a comma
#             item = div.text.strip().split(',', 1)[0]
            
#             #removes the word '(optional)' which appears next to some ingredients
#             item = item.split('(', 1)[0]
            
#             #add url, recipe, and ingredient to their corresponding list
#             ingredientList.append(item.lower())
#             recipeURL.append(link.lower())
#             recipeName.append(name.lower())
        
       

In [ ]:
# rawData = pd.DataFrame(
#     {'url': recipeURL,
#      'name': recipeName,
#      'ingredient': ingredientList
#     })

# rawData.to_csv('SUN_info370_assignment1_rawData.csv')